In [14]:
import pandas as pd
import numpy as np

In [15]:
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')

In [16]:
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [17]:
credits.head(1)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [18]:

movies = movies.merge(credits,on='title')

movies.head(1)

movies.info()

movies['original_language'].value_counts()




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4809 entries, 0 to 4808
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4809 non-null   int64  
 1   genres                4809 non-null   object 
 2   homepage              1713 non-null   object 
 3   id                    4809 non-null   int64  
 4   keywords              4809 non-null   object 
 5   original_language     4809 non-null   object 
 6   original_title        4809 non-null   object 
 7   overview              4806 non-null   object 
 8   popularity            4809 non-null   float64
 9   production_companies  4809 non-null   object 
 10  production_countries  4809 non-null   object 
 11  release_date          4808 non-null   object 
 12  revenue               4809 non-null   int64  
 13  runtime               4807 non-null   float64
 14  spoken_languages      4809 non-null   object 
 15  status               

original_language
en    4510
fr      70
es      32
zh      27
de      27
hi      19
ja      16
it      14
ko      12
cn      12
ru      11
pt       9
da       7
sv       5
nl       4
fa       4
th       3
he       3
ta       2
cs       2
ro       2
id       2
ar       2
vi       1
sl       1
ps       1
no       1
ky       1
hu       1
pl       1
af       1
nb       1
tr       1
is       1
xx       1
te       1
el       1
Name: count, dtype: int64

In [19]:
movies.isnull().sum()

movies.dropna(inplace = True)



In [20]:
movies.isnull().sum()

movies.iloc[0].genres

movies = movies[['movie_id','title','overview','original_language','genres','keywords','cast','crew']]



In [21]:
import ast
def convert(obj):
  L = []
  for i in ast.literal_eval(obj):
    L.append(i['name'])
    return L

movies['genres'] = movies['genres'].apply(convert)

movies['keywords'] = movies['keywords'].apply(convert)




In [22]:
import ast

def convert3(obj):
    L = []
    counter = 0
    for i in ast.literal_eval(obj):
        if counter != 3:
            L.append(i['name'])
            counter += 1
        else:
            break
    return L

movies['cast'] = movies['cast'].apply(convert3)

movies['cast']

movies.head(1)

movies['crew'][0] #will extract the director

def fetch_director(text):
    L = []
    for i in ast.literal_eval(text):
        if i['job'] == 'Director':
            L.append(i['name'])
    return L

movies['crew'] = movies['crew'].apply(fetch_director)

movies['crew']

movies.head()

"""Our main motive is to extract main points and convert the things into list"""

movies['overview'] = movies['overview'].apply(lambda x:x.split())

movies['original_language'] = movies['original_language'].apply(lambda x:x.split())

movies.head()

movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

movies.head()

new = movies.drop(columns=['overview','genres','keywords','cast','crew'])

new_df = movies[['movie_id','title','tags']]

new_df

new_df['tags'][0]

new_df['tags'] = [' '.join(tags) if isinstance(tags, list) else str(tags) for tags in new_df['tags']]

from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()

tfidf_matrix = tfidf.fit_transform(new_df['tags'])

tfidf_matrix

from sklearn.metrics.pairwise import cosine_similarity

similarity = cosine_similarity(tfidf_matrix)

similarity[1]

def recommend(movie):
    # Get the index of the movie that matches the title
    index = new_df[new_df['title'] == movie].index[0]

    # Get the pairwise similarity scores of all movies with that movie
    distances = similarity[index]

    # Enumerate over the distances and sort them in descending order of similarity
    # Skip the first one as it's the movie itself
    movie_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:6]

    # Print the titles of the top 5 recommended movies
    for i in movie_list:
        print(new_df.iloc[i[0]].title)

recommend('Batman Begins')

new_df.iloc[1216].title


Kung Fu Panda 2
Kung Fu Panda
X-Men: Days of Future Past
Larry Crowne
Jack the Giant Slayer


C:\Users\DELL\AppData\Local\Temp\ipykernel_8344\182843698.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = [' '.join(tags) if isinstance(tags, list) else str(tags) for tags in new_df['tags']]


'The Words'

In [30]:
import pickle

In [31]:
pickle.dump(new_df,open('movies.pkl','wb'))

In [32]:
new_df['title'].values

array(['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre',
       ..., 'Tin Can Man', 'Primer', 'Shanghai Calling'], dtype=object)

In [33]:
pickle.dump(new_df.to_dict(),open('movie_dict.pkl','wb'))

In [34]:
pickle.dump(similarity,open('similarity.pkl', 'wb'))